#### IMPORTANT
Ensure you are utilizing 64-bit REFPROP with 64-bit python. If using the free version of REFPROP (MINI-REFPROP), please use 32-bit python and make changes to match the location where MINI-REFPROP is installed and make changes to the REFPROPFunctionLibrary function to read the REFPROP.DLL file.

Information on REFPROP and functions can be found here: https://buildmedia.readthedocs.org/media/pdf/refprop-docs/latest/refprop-docs.pdf

### IMPORT PACKAGES & FUNCTIONS

In [1]:
# Dictate the environment's loctaion of REFPROP
import os
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'

In [2]:
# Import the main class from the Python library
from ctREFPROP.ctREFPROP import REFPROPFunctionLibrary

# Imports from conda-installable packages
import pandas as pd

# Import numpy
import numpy as np

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Import Math for common values such as PI
import math


In [3]:
# Instantiate the library, and use the environment variable to explicitly state which path we want to use.
# As mentioned above, this will be changed to call the correct REFPROP functions to be used
# with MINI-REFPROP and 32-bit python.
# If using MINI-REFPROP and 32-bit python please make the following changes
# RP = REFPROPFunctionLibrary('C:/Program FIles (x86)/MINI-REFPROP\\REFPROP.DLL')
RP = REFPROPFunctionLibrary(os.environ['RPPREFIX'])

In [4]:
# This will call which root directory that will be used for the program. 
RP.SETPATHdll(os.environ['RPPREFIX'])

In [5]:
# Get the unit system we want to use (Mass base SI gives units in
# K, Pa, kg, m, N, J, W, and s)
MASS_BASE_SI = RP.GETENUMdll(0, "MASS BASE SI").iEnum

### sCO2 Loop Calculations

#### System Parameters

In [6]:
# Tube inner diameter and outer diameter
Tube_OD = 0.50 # [inch]
Tube_Thick = 0.065 # [inch]
Tube_ID = Tube_OD - 2 * Tube_Thick # inch

Tube_OD = Tube_OD * 0.0254 # [Convert inches to meters]
Tube_ID = Tube_ID * 0.0254 # [Convert inches to meters]

# Mass flow rate of sCO2
m_dot = 0.2 # [kg/s]

#### Outlet of Compressor (State  1)

In [7]:
# Temperature will be compared at end of script
T1 = 60 # [C]
P1 = 2500 # [psia]

T1 = T1 + 273.15 # Convert C to Kelvin
P1 = P1 * 6894.8 # convert psia to Pa

print("Pressure at Outlet of Compressor =", P1/6894.8, "psia")
print("Temperature at Outlet of Compressor =" , (T1 - 273.15) * (9/5) + 32, "F")

Pressure at Outlet of Compressor = 2500.0 psia
Temperature at Outlet of Compressor = 140.0 F


In [8]:
# Obtain fluid properties from the pressure and temperature outlined above
State_1 = RP.REFPROPdll("CO2","PT","D;V;H;S;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P1,T1,[1.0])

# Outputs will be placed into data frame for organization
State_1 = pd.DataFrame(State_1.Output[0:9],
            index = ['Density [kg/m^3]', 'Volume [m^3/kg]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 1'])

# Display the data frame
State_1

,State 1
Density [kg/m^3],670.303564
Volume [m^3/kg],0.001492
Enthalpy [J/kg],333792.938415
Entropy [J/kg],1386.087141
CP/CV,3.061084
Speed of Sound,359.912293
Thermal Cond. [W/(mK)],0.072382
Viscosity [Pa-s],0.000053
Prandtl,2.115600


In [9]:
# Find Velocity, Reynolds Number, and darcy friction factor (assuming smooth pipe)
Velocity = 4 * m_dot * State_1.loc['Volume [m^3/kg]','State 1'] / (math.pi * Tube_ID**2)
Reynolds = State_1.loc['Density [kg/m^3]','State 1'] * Velocity * Tube_ID / State_1.loc['Viscosity [Pa-s]','State 1']
Darcy_f = (0.79 * math.log(Reynolds) - 1.64)**(-2)

In [10]:
# Using Estimated Length of Tubing connecting compressor and Heat Source, 
# find the amount of pressure drop caused by fanno flow
Length = 3.71475 # [meters]

# Force acted on the wall of tube
Force = math.pi * Tube_ID * Darcy_f * State_1.loc['Density [kg/m^3]','State 1'] * (Velocity**2) * Length / 8 

# Dimensionless Friction factor
f_dim = 4 * Force / (P1 * math.pi * Tube_ID**2) 

# Inlet Mach Number of length of tubing
Mach_inlet = Velocity / State_1.loc['Speed of Sound','State 1']

# Formulation used to calculate Pressure drop (found in Adv. Fluid Mechanics Textbook)
A_eq = ((Mach_inlet**2) * (1 + ((State_1.loc['CP/CV','State 1'] - 1) / 2) * (Mach_inlet**2))) \
        / ((1 + State_1.loc['CP/CV','State 1'] * (Mach_inlet**2) - f_dim)**2)

# Either Mach numbers will be negative. Use the positive value in the following calculations
Mach_outlet_1 = math.sqrt((-1 * (1 - 2 * A_eq * State_1.loc['CP/CV','State 1'])\
                          + ((1 + 2 * A_eq * (State_1.loc['CP/CV','State 1'] + 1))**0.5))\
                         / ((State_1.loc['CP/CV','State 1'] - 1) - 2 * A_eq * State_1.loc['CP/CV','State 1']**2))
#Mach_outlet_2 = math.sqrt((-1 * (1 - 2 * A_eq * State_1.loc['CP/CV','State 1'])\
#                          - ((1 + 2 * A_eq * (State_1.loc['CP/CV','State 1'] + 1))**0.5))\
#                         / ((State_1.loc['CP/CV','State 1'] - 1) - 2 * A_eq * State_1.loc['CP/CV','State 1']**2))

# Find Outlet pressure caused by fanno flow (frictional loss)
P2 = P1 * (1 + State_1.loc['CP/CV','State 1'] * (Mach_inlet**2) - f_dim) / (1 + State_1.loc['CP/CV','State 1']\
                                                                           * (Mach_outlet_1**2))

print("Pressure at Inlet of Heat Source =", round(P2/6894.8 , 3), "psia")
print("Temperature at Inlet of Heat Source =" , (T1 - 273.15) * (9/5) + 32, "F")

Pressure at Inlet of Heat Source = 2491.034 psia
Temperature at Inlet of Heat Source = 140.0 F
